In [70]:
import pandas as pd 
import ast
from bertopic import BERTopic
from collections import Counter
import random 
import random
import uuid
from datetime import datetime, timedelta
import json


In [57]:
casa_hotels = pd.read_csv('edited-copy-of-hotels-in-casablanca.csv')
doha_hotels = pd.read_csv('edited-copy-of-hotels-in-doha.csv')
marrakech_hotels = pd.read_csv('edited-copy-of-hotels-in-marrakech.csv')
casa_rest = pd.read_csv('edited-copy-of-restaurants-in-casablanca.csv')
doha_rest = pd.read_csv('edited-copy-of-restaurants-in-doha.csv')
marrakech_rest = pd.read_csv('edited-copy-of-restaurants-in-marrakech.csv')


hotels  = pd.concat([casa_hotels, doha_hotels, marrakech_hotels], ignore_index=True)
resturants = pd.concat([casa_rest, doha_rest, marrakech_rest], ignore_index=True)
hotels['review_keywords'] = hotels['review_keywords'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
resturants['cuisines'] = resturants['cuisines'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
resturants['diets'] = resturants['diets'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
resturants['meal_types']  = resturants['meal_types'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
resturants['dining_options'] = resturants['dining_options'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
resturants['top_tags'] = resturants['top_tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

key_words = pd.concat([
    hotels['review_keywords'],resturants[['cuisines','diets','meal_types','dining_options','top_tags']]
], ignore_index=True).stack()
key_words = key_words.dropna() 
key_lists =  [word for lis in key_words if  isinstance(lis,list) for word in lis]
key_lists = list(set(key_lists))  

 


In [ ]:
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(key_lists)
#topic_model.reduce_topics(key_lists, nr_topics=5)




In [ ]:
topic_info = topic_model.get_topic_info()
topics = pd.DataFrame(topic_info[['Topic','Representative_Docs','Representation']])
words_topic = topic_model.get_document_info(key_lists).groupby('Topic')['Document'].apply(list).reset_index()
topic_model.get_document_info(key_lists)

In [5]:

topic_mapping = {
    -1: {"name": "General Experience", "tags": [ "parking", "shuttle", "experience"]},
    0: {"name": "Cultural & Language", "tags": ["Oreintal", "Cultural", "Arabic","Markets & Souks", "Heritage","Outdoor Spaces"]},
    1: {"name": "Dining & Food", "tags": ["restaurant", "buffet",  'Dining',"Food & Drink",  "Cuisine", "Beverages",  "Local Flavors"]},
    2: {"name": "Hotel Experience", "tags": [ "Luxury Stays","5-Star Comfort","World-Class Amenities","Elegant Retreats","Premium Hospitality","Opulent Escapes","Scenic Getaways"]},
    3: {"name": "Pools & Water Activities", "tags": [ "Swimming & Pools","Water Activities","Resort & Relaxation","Refreshing Retreats","Outdoor Spaces","Luxury"]},
    4: {"name": "Hotel Staff & Service", "tags": ["Hospitality","Hotel Staff","Guest Experience", 'Accommodation']},
    5: {"name": "Hotel Staff & Service", "tags": ["Hospitality","Hotel Staff","Guest Experience", "Comfort & Hygiene"]},
    6: {"name": "Long-term Stay", "tags": ["long stay","Guest Experience","Comfort & Enjoyment","Positivity"]},
    7: {"name": "Nightlife & Entertainment", "tags": ["Nightlife","Evening Activities","Late Night Experience"]},
    8: {"name": "Cleaning & Housekeeping", "tags": ["Accommodation","Guest Experience","Comfort & Hygiene",'Hospitality']},
    9: {"name": "City & Location", "tags": ["City Vibes","Hidden Gems","Peace & Quiet","Prime Location","Easy Access","Bustling Streets"]},
    10: {"name": "Bars & Lounges", "tags": ["Vibrant Nightlife","Live Jazz & Blues","Skyline Views","Sports & Spirits","Chic Cocktail Bars","Ocean Bites","Lounge & Leisure"]},
    11: {"name": "Lounges & Clubs", "tags": ["lounge","Luxury Lounge","Chill & Relax","Exclusive Club","Vibrant Pub Vibes","Accessible Leisure","sunbeds"]},
    12: {"name": "Transportation", "tags": ["airport", "taxi", "transit", "luggage"]},
    13: {"name": "Tours & Walking", "tags": ["walking tour", "guided tour","Guided Adventures","Scenic Walks","Short Getaways","Excursion Ready","Walk & Discover"]},
    14: {"name": "Relaxing Holidays", "tags": ["relaxing", "holiday", "Vacation Bliss","Perfect Week","Time Well Spent","Amazing Adventures"]},
    15: {"name": "Wellness & Spiritual", "tags": ["Positive Vibes","Mindful","Spiritual","Wellness & Balance","Supportive Atmosphere",]},
    16: {"name": "Culturals & Languages", "tags": ["Cultural Fusion","Arabian Nights","Marrakech Magic","Souk Adventures","Persian Vibes","Qatari Elegance","French Flair","Authentic Traditions","Jemaa El-Fnaa Experience","Multilingual Charm"
]}
}


In [11]:

df = pd.DataFrame.from_dict(topic_mapping, orient="index").reset_index()
topic_info['Tags_info'] = topic_info['Tags_info'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
topic_info['Document_new'] = topic_info['Document_new'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

topic_info['topic_name'] = topic_info['Topic'].map(lambda x: topic_mapping[x]["name"] if x in topic_mapping else "Unknown")
df.columns = ["Topic", "Name", "Tags"]
topic_ = topic_info.merge(df, on="Topic", how="inner", suffixes=("_inf", "_new"))
topic_info = topic_.merge(words_topic, on="Topic", how="inner", suffixes=("_inf", "_new"))
topic_info.to_csv("topic_info_docu.csv", index=False)

topic_info.head()


,Topic,Count,Name_info,Representation,Representative_Docs,Name_new,Tags_info,Document_info,Name,Tags_new,Document_new,topic_name
0,-1,211,-1_riad_the_experience_parking,"['riad', 'the', 'experience', 'parking', 'bed'...","['the hotel shuttle bus', 'away from the hustl...",General Experience,"[riad, parking, shuttle, experience]","['very nice experience', 'jacuzzi', 'early che...",General Experience,"['riad', 'parking', 'shuttle', 'experience']","[very nice experience, jacuzzi, early check, m...",General Experience
1,0,189,0_moroccan_french_arabic_el,"['moroccan', 'french', 'arabic', 'el', 'marrak...","['French', 'french', 'Moroccan']",Cultural & Language,"[Oreintal, Cultural, Arabic, Markets & Souks, ...","['bassem', 'bira', 'huda', 'asif', 'African', ...",Cultural & Language,"['Oreintal', 'Cultural', 'Arabic', 'Markets & ...","[bassem, bira, huda, asif, African, omar, flex...",Cultural & Language
2,1,79,1_restaurant_breakfast_buffet_cafe,"['restaurant', 'breakfast', 'buffet', 'cafe', ...","['breakfast buffet', 'buffet restaurant', 'bre...",Dining & Food,"[restaurant, buffet, Dining, Food & Drink, Cui...","['semi buffet', 'Pizza', 'Gluten free options'...",Dining & Food,"['restaurant', 'buffet', 'Dining', 'Food & Dri...","[semi buffet, Pizza, Gluten free options, mint...",Dining & Food
3,2,50,2_hotel_great_this_is,"['hotel', 'great', 'this', 'is', 'beautiful', ...","['beautiful hotel', 'hotel is beautiful', 'gre...",Hotel Experience,"[Luxury Stays, 5-Star Comfort, World-Class Ame...","['would highly recommend this hotel', 'amazing...",Hotel Experience,"['Luxury Stays', '5-Star Comfort', 'World-Clas...","[would highly recommend this hotel, amazing ho...",Hotel Experience
4,3,40,3_pool_aqua_area_pools,"['pool', 'aqua', 'area', 'pools', 'aerobics', ...","['pool bar', 'pool area', 'pool']",Pools & Water Activities,"[Swimming & Pools, Water Activities, Resort & ...","['pool games', 'aqua zumba', 'pool area is lov...",Pools & Water Activities,"['Swimming & Pools', 'Water Activities', 'Reso...","[pool games, aqua zumba, pool area is lovely, ...",Pools & Water Activities


In [ ]:
def most_frequent(arr):
    return max(set(arr), key=arr.count) if arr else None 
def assign_tags_and_topics(col):
    tags_list = []
    topic_list = []
    for row in col:
        row_set = set(row) if isinstance(row,list) else set()
        selected_tags = []
        selected_topics = []
        for tag, lis, topic in zip(topic_info["Tags_info"], topic_info["Document_new"],topic_info['topic_name']):
            word_info = set(tag)
            lis_set = set(lis)
            if not row_set.isdisjoint(lis_set):  
                num_tags = random.randint(0,1)
                selected_tags.extend(random.sample(tag, num_tags))
                selected_topics.append(topic)
        topic_list.append(most_frequent(selected_topics))
        tags_list.append(selected_tags if selected_tags else [])
    return tags_list, topic_list

intervel = hotels["current_rank"].max() - hotels["current_rank"].min() / 3
step = (hotels["current_rank"].max() - hotels["current_rank"].min()) // 3  
low_max = hotels["current_rank"].min() + step  
mid_max = low_max + step  
luxury_min = mid_max + 1  

def categorize_price(rank):
    if rank <= low_max:
        return "low"
    elif rank <= mid_max:
        return "mid-range"
    else:
        return "luxury"

In [141]:
resturants = pd.read_csv('/home/alaa/project/restaurants_final.csv')
resturants['cuisines'] = resturants['cuisines'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
resturants['diets'] = resturants['diets'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
resturants['meal_types']  = resturants['meal_types'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
resturants['dining_options'] = resturants['dining_options'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
resturants['top_tags'] = resturants['top_tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
hotels = pd.read_csv('/home/alaa/project/hotels_final.csv')
hotels['review_keywords'] = hotels['review_keywords'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
city_country = {'Marrakech':"Morocco",
                'Casablanca':"Morocco",
                'Doha':'Qatar'}
merge_column = []


for _, row in resturants[['cuisines', 'diets', 'meal_types', 'dining_options', 'top_tags']].iterrows():
    merged_row = []

    for col in ['cuisines', 'diets', 'meal_types', 'dining_options', 'top_tags']:
        value = row[col]  
        if not isinstance(value, list):
            value = [value] if pd.notna(value) else []  
        merged_row.extend(value)
    
    merge_column.append(merged_row) 

resturants['merged_keywords'] = merge_column
resturants['merged_keywords'] = resturants['merged_keywords'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
resturants['country'] = resturants['city'].map(city_country)
hotels['country'] = hotels['city'].map(city_country)
hotels["TAG"], hotels["Topic"] = assign_tags_and_topics(hotels["review_keywords"]) 
resturants["TAG"], resturants["Topic"] = assign_tags_and_topics(resturants["merged_keywords"])
resturants['price_range'] = resturants['price_range'].astype(str).apply(lambda x: x.split('-')[-1].strip())

hotels["current_rank"] = hotels["ranking"].apply(lambda x: json.loads(x)["current_rank"])


hotels["price_range"] = hotels["current_rank"].apply(categorize_price)

price_mapping = {"$": 'low', "$$": "mid-range", "$$$": "mid-range", "$$$$": "luxury"}
resturants["price_range"] = resturants["price_range"].map(price_mapping)
resturants["price_range"].fillna(resturants["price_range"].mode(), inplace=True)
hotels['description']  = hotels['review_keywords']

intesected_columns = resturants.columns.intersection(hotels.columns)

restaurants_common = resturants[intesected_columns]
hotels_common = hotels[intesected_columns]

destinations = pd.concat([restaurants_common, hotels_common], ignore_index=True)
destinations = destinations.drop(columns=['Unnamed: 0', 'id', 'email', 'phone', 'link'], errors='ignore')

destinations.to_csv('destinations.csv')


In [169]:
trip_durations = ['day','3 days', 'week','2 weeks','month','more than a month']

group_sizes = ['solo', 'couple', 'family', 'group', 'big-group']

description_templates = {
    "General Experience": [
        ("Unforgettable Experiences in {city}", "Enjoy a well-rounded experience in {city} with amazing attractions and activities."),
        ("Discover {city}: A Journey Through Unique Moments", "From historic sites to modern attractions, explore everything {city} has to offer."),
        ("The Ultimate {city} Getaway", "Plan the perfect trip filled with diverse experiences, from sightseeing to local adventures."),
    ],
    "Cultural & Language": [
        ("Cultural Treasures of {city}", "Immerse yourself in the traditions, languages, and heritage of {city}."),
        ("A Walk Through {city}'s History", "Explore ancient landmarks, traditional markets, and vibrant local culture."),
        ("Heritage & Language Experience in {city}", "Engage with the rich cultural diversity and learn about local languages and traditions."),
    ],
    "Dining & Food": [
        ("Foodie Tour in {city}", "Savor the best local dishes and gourmet experiences {city} has to offer."),
        ("Culinary Delights of {city}", "Explore traditional flavors, street food gems, and fine dining spots."),
        ("Taste the Authentic {city} Experience", "From markets to Michelin-star restaurants, indulge in a gastronomic journey."),
    ],
    "Hotel Experience": [
        ("Luxury Retreat in {city}", "Experience world-class hospitality in the finest hotels of {city}."),
        ("5-Star Stay in {city}", "Unwind in top-rated hotels featuring premium amenities and stunning views."),
        ("The Ultimate Hotel Experience in {city}", "Relax in elegant accommodations with top-tier services and comfort."),
    ],
    "Pools & Water Activities": [
        ("Water Adventures in {city}", "Swim, surf, and enjoy thrilling water activities in {city}."),
        ("Luxury Poolside Relaxation in {city}", "Soak in the sun and unwind at the best pools and resorts."),
        ("Diving & Snorkeling Escape in {city}", "Explore the underwater wonders and crystal-clear waters of {city}."),
    ],
    "Nightlife & Entertainment": [
        ("Vibrant Nights in {city}", "Enjoy electrifying nightlife, clubs, and live entertainment in {city}."),
        ("The Best Nightlife Hotspots in {city}", "Party all night at the most exciting bars and clubs."),
        ("Live Music & Dance in {city}", "Experience unforgettable live music, dance shows, and performances."),
    ],
    "City & Location": [
        ("Hidden Gems of {city}", "Discover the most charming and lesser-known attractions in {city}."),
        ("City Vibes & Urban Adventures in {city}", "Experience the energy of {city} with its lively streets and cultural hotspots."),
        ("Exploring {city}: The Best Spots to Visit", "From historic sites to modern attractions, explore everything {city} has to offer."),
    ],
    "Tours & Walking": [
        ("Scenic Walking Tours in {city}", "Stroll through the picturesque streets and historic districts of {city}."),
        ("Guided Tours & Excursions in {city}", "Join an expert guide to uncover the fascinating stories of {city}."),
        ("Adventure on Foot: Exploring {city}", "Discover hidden alleyways, markets, and cultural sites on a walking tour."),
    ],
    "Relaxing Holidays": [
        ("The Perfect Holiday in {city}", "Enjoy a peaceful and relaxing vacation with stunning scenery and cozy accommodations."),
        ("Unwind in {city}: A Relaxation Escape", "Recharge your energy with the best spa, wellness, and leisure experiences."),
        ("Serene Retreat in {city}", "Escape the hustle and bustle and enjoy a slow, peaceful vacation."),
    ],
    "Wellness & Spiritual": [
        ("Wellness & Mindfulness in {city}", "Find balance and relaxation with yoga retreats, wellness spas, and spiritual experiences."),
        ("Spiritual Journeys in {city}", "Discover sacred sites, meditation retreats, and positive energy in {city}."),
        ("A Healing Getaway in {city}", "Enjoy a rejuvenating experience with wellness-focused activities in {city}."),
    ],
}
def generate_trip_name_and_description(topics, city, country):
    """Generates a trip name & description based on the most common topic."""
    if not topics:
        return f"Exploring {city}", f"Discover the best places and attractions in {city}."
    
    sampled_topics = random.sample(topics, min(len(topics), random.randint(1, min(5, len(topics)))))  # Ensure at least one topic
    most_common_topic = Counter(sampled_topics).most_common(1)[0][0] if sampled_topics else None  

    if most_common_topic and most_common_topic in description_templates:
        name_template, desc_template = random.choice(description_templates[most_common_topic])
        return name_template.format(city=city, country=country), desc_template.format(city=city, country=country)

    return f"Exploring {city}", f"Discover the best places and attractions in {city}."


    return

def most_frequent_trip_tags(arr):
    """Finds the most common tag appearing between 2 and 5 times."""
    if not arr:
        return None  
    counts = Counter(arr)
    valid_items = {key: value for key, value in counts.items() if 2 <= value <= 5}

    return max(valid_items, key=valid_items.get) if valid_items else None

def select_duration(destination_count):
    """Selects trip duration based on the number of destinations (2-7)."""
    if destination_count <= 3:
        return random.choice(['3 days', 'week'])
    elif destination_count <= 5:
        return random.choice(['week', '2 weeks'])
    else:
        return random.choice(['2 weeks', 'month'])

In [170]:
def generate_trip(destinations):
    
    trips = []

    trip_groups = destinations.groupby('city')

    for city, group in trip_groups:
        country = group['country'].iloc[0]
        topics = group['Topic'].dropna().tolist()
        trip_name, trip_desc = generate_trip_name_and_description(topics, city, country)
        group['rating'] = pd.to_numeric(group['rating'], errors='coerce') 
        selected_destinations = group.sort_values(by="rating", ascending=False).head(random.randint(2, 5))
        tags = [tag for sublist in selected_destinations["TAG"] if isinstance(sublist, list) for tag in sublist]
        most_common_tag = most_frequent_trip_tags(tags)
        price_list = selected_destinations["price_range"].dropna().tolist()
        price_range = most_frequent(price_list) if price_list else "Unknown"
        destination_count = len(selected_destinations)
        duration = select_duration(destination_count)
        group_size = random.choice(group_sizes)

        # Trip metadata
        trip = {
            "id": str(uuid.uuid4()),  # Generate a unique ID
            "title": trip_name,
            "description": trip_desc,
            "country": country,
            "destinationCount": destination_count,
            "tags": tags,
            "popular_tag": most_common_tag,
            "rating": round(selected_destinations["rating"].mean(), 1) if not selected_destinations.empty else 0,
            "price_range": price_range,
            "destinations": list(selected_destinations["name"]),
            "coverImage": f"{city.lower().replace(' ', '_')}.jpg",  # Placeholder image
            "cloneCount": 1,
            "sameCountry": True,
            "creationDate": datetime.today().strftime('%Y-%m-%d'),
            "lastEditDate": datetime.today().strftime('%Y-%m-%d'),
            "wishedCount": 0,
            "favoriteCount": 0,
            "trip_duration": duration,
            "group_size": group_size,
        }

        trips.append(trip)

    return trips

In [174]:
trips = generate_trip(destinations)
trips_df = pd.DataFrame(trips)

# Display the generated trips
trips_df.to_csv('trips.csv')

In [5]:
import pandas as pd 
import numpy as np 
 
group_sizes = ['solo', 'couple', 'family', 'group', 'big-group']
data =  pd.read_csv('/home/alaa/recommender_system/database/df_cleaned.csv')
data['group_size']  = np.random.choice(group_sizes, size=len(data))
data.to_csv('destinations_final.csv')

In [6]:
data

,Unnamed: 0,name,description,reviews,rating,website,featured_image,ranking,address,latitude,longitude,reviews_per_rating,review_keywords,price_range,city,destinationType,TAG,country,Topic,group_size
0,2,Pointbar,"In a quiet street of Gueliz is the Pointbar, a...",1818,5.0,http://www.pointbar-restaurant.com,https://media-cdn.tripadvisor.com/media/photo-...,"{""current_rank"": 3, ""total"": 1580}","3 bis rue Abou Hayane, Marrakech 40000 Morocco",31.639067,-8.014753,"{""1"": 22, ""2"": 10, ""3"": 20, ""4"": 111, ""5"": 1655}","[""mussels"", ""gyoza"", ""tapas"", ""steak"", ""our wa...",mid-range,Marrakech,Restaurant,"['Cultural', 'buffet', 'Hospitality']",Morocco,General Experience,group
1,3,La Terrasse De La Brillante - Rooftop,The restaurant is located on the rooftop of th...,642,5.0,http://www.laterrassedelabrillante.com/,https://media-cdn.tripadvisor.com/media/photo-...,"{""current_rank"": 9, ""total"": 1580}","Rue Bahia 6, Marrakech 40000 Morocco",31.622390,-7.984030,"{""1"": 3, ""2"": 1, ""3"": 12, ""4"": 27, ""5"": 599}","[""tuna"", ""bahia palace"", ""modern setting"", ""gr...",luxury,Marrakech,Restaurant,"['Cuisine', 'Mindful']",Morocco,General Experience,couple
2,4,Palais Dar Soukkar,Palais Dar Soukkar offers to its guests the po...,2585,4.5,http://www.darsoukkar.com,https://media-cdn.tripadvisor.com/media/photo-...,"{""current_rank"": 80, ""total"": 1580}","Km 3,5 Route de l'Ourika, Marrakech 40000 Morocco",31.574932,-7.978856,"{""1"": 152, ""2"": 65, ""3"": 60, ""4"": 200, ""5"": 2108}","[""cake"", ""amazing entertainment"", ""amazing nig...",mid-range,Marrakech,Restaurant,"['Arabic', 'Local Flavors', 'Easy Access', 'Oc...",Morocco,Bars & Lounges,couple
3,9,NOMAD,"In 2014, Kamal Laftimi and Sebastian de Gzell ...",6724,4.0,http://nomadmarrakech.com,https://media-cdn.tripadvisor.com/media/photo-...,"{""current_rank"": 264, ""total"": 1580}","1, Derb Aajrane, Marrakech 40000 Morocco",31.628593,-7.987534,"{""1"": 251, ""2"": 444, ""3"": 988, ""4"": 2094, ""5"":...","[""lamb burger"", ""courgette fritters"", ""cake"", ...",mid-range,Marrakech,Restaurant,"['parking', 'Dining', 'Late Night Experience',...",Morocco,Bars & Lounges,group
4,11,Azar,Lebanese and moroccan restaurant located in Gu...,3625,4.5,https://www.facebook.com/AzarMarrakech/,https://media-cdn.tripadvisor.com/media/photo-...,"{""current_rank"": 79, ""total"": 1580}","Rue de Yougoslavie, Marrakech Morocco",31.631748,-8.012902,"{""1"": 68, ""2"": 48, ""3"": 189, ""4"": 715, ""5"": 2605}","[""lamb shawarma"", ""mixed grill"", ""tasting menu...",luxury,Marrakech,Restaurant,"['shuttle', 'buffet', 'Exclusive Club']",Morocco,Bars & Lounges,group
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,141,Idou Anfa Hotel,"['top floor', 'hassan ii', 'the upper floors',...",93,4.0,NaN,https://dynamic-media-cdn.tripadvisor.com/medi...,"{""current_rank"": 34, ""total"": 121}","85 Boulevard d'Anfa, Casablanca 20000 Morocco",33.592255,-7.632555,"{""1"": 9, ""2"": 6, ""3"": 9, ""4"": 33, ""5"": 36}","['top floor', 'hassan ii', 'the upper floors',...",low,Casablanca,Hotel,"['Luxury Stays', 'Hotel Staff', 'Hidden Gems']",Morocco,City & Location,group
119,142,Kenzi Tower Hotel,"['sky bar', 'top floor', 'business trip', 'rai...",66,4.0,NaN,https://dynamic-media-cdn.tripadvisor.com/medi...,"{""current_rank"": 38, ""total"": 121}","Bd Mohammed Zerktouni Twin Center, Casablanca ...",33.586864,-7.631972,"{""1"": 7, ""2"": 5, ""3"": 11, ""4"": 8, ""5"": 35}","['sky bar', 'top floor', 'business trip', 'rai...",low,Casablanca,Hotel,"['parking', 'Bustling Streets']",Morocco,Bars & Lounges,big-group
120,143,Unico Hotel,[],9,4.0,NaN,https://dynamic-media-cdn.tripadvisor.com/medi...,"{""current_rank"": 72, ""total"": 121}","5 Rue du Gabon, Casablanca 20250 Morocco",33.592800,-7.634710,"{""1"": 2, ""2"": 1, ""3"": 0, ""4"": 0, ""5"": 6}",[],low,Casablanca,Hotel,[],Morocco,NaN,solo
121,144,Casablanca Le Lido Thalasso & Spa,"['hassan ii', 'mosque']",611,3.0,N